# **`Common class`**


In [1]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import keras, math
from keras.callbacks import LearningRateScheduler
from keras import backend as K


class Common:
     
    def __init__(self, initial_lrate=0.1, drop=0.8, epochs_drop=0.1, limit_inf=0.001):
        self.initial_lrate = initial_lrate
        self.drop = drop
        self.epochs_drop = epochs_drop
        self.limit_inf = limit_inf
        self.model = None
        
    def step_decay(self, epoch):
        print("[+] initial: ", self.initial_lrate)
        print("[+] ANTIGO K: ", K.eval(self.model.optimizer.lr))

        if epoch == 0:
            lrate = self.initial_lrate
        else:
            lrate = self.initial_lrate * math.pow(self.drop, math.floor((1+epoch)/self.epochs_drop))
            if lrate <= self.limit_inf:
                lrate = self.limit_inf            
        print("[+] NOVO: ", lrate)
        return lrate

    def save_plot_losses(self, history, path_dir):
        plt.figure(figsize=[8,6])
        plt.plot(history.history['loss'],'r',linewidth=3.0)
        plt.plot(history.history['val_loss'],'b',linewidth=3.0)
        plt.legend(['Custo no Treino', 'Custo no Teste'],fontsize=18)
        plt.xlabel('Épocas ',fontsize=16)
        plt.ylabel('Custo',fontsize=16)
        plt.title('Curvas da Função de Custo',fontsize=16)
        plt.savefig(os.path.join(path_dir, 'loss.png'))

    def save_plot_acc(self, history, path_dir):
        plt.figure(figsize=[8,6])
        plt.plot(history.history['acc'],'r',linewidth=3.0)
        plt.plot(history.history['val_acc'],'b',linewidth=3.0)
        plt.legend(['Acurácia no Treino', 'Acurácia no Teste'],fontsize=18)
        plt.xlabel('Épocas ',fontsize=16)
        plt.ylabel('Acurácia',fontsize=16)
        plt.title('Curvas da Acurácia',fontsize=16)
        plt.savefig(os.path.join(path_dir, 'acc.png'))

    def write_file_info(self, path_dir, content):
        file_info_path = os.path.join(path_dir, "info.txt")
        file_info_experiment = open(file_info_path, "a+")

        for i in content:
            line = str(i) + "\n"
            file_info_experiment.write(line)

        file_info_experiment.close()

    def addCommentary(self, path_dir, commentary=""):
        self.write_file_info(path_dir, ["commentary: " + commentary])
        
    def train(self, model, x_train, y_train, x_test, y_test, batch_size, epochs, path_dir, patience):
        self.write_file_info(path_dir, ["batch_size: "+str(batch_size)])
        
        self.model = model

        file_weights = os.path.join(path_dir, "weights.{epoch:02d}-{val_loss:.2f}.hdf5")
        checkpoint = ModelCheckpoint(file_weights, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
#         checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=1, mode='auto')
        
        lrate = LearningRateScheduler(self.step_decay)
        
        callbacks_list = [checkpoint, earlystopping, lrate]
        print("[+] BATCH_SIZE: ", batch_size)
        history = self.model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
#                     steps_per_epoch = 100,
                    shuffle = True,
                    validation_data=(x_test, y_test),
                    callbacks = callbacks_list)

        return self.model, history

    def evaluate(self, model, x_test, y_test):
        score = model.evaluate(x_test, y_test, verbose=0)
        return score

    def compile_net(self, model, path_dir, option):

        self.write_file_info(path_dir, ["learning_rate_initial: "+str(self.initial_lrate), \
                                        "learning_rate_inf: "+str(self.limit_inf), \
                                        "learnig_rate_drop: "+ str(self.drop),\
                                        "learnig_rate_epochs_drop: "+ str(self.epochs_drop),\
                                        "optimizer: " + str(option)])

#         print("[+] summary network:")
#         print(model.summary())
        
        self.model = model
        print("[+] OPTIMIZER: ", option)       
        
        if option == "Adadelta":
            self.model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                        optimizer=keras.optimizers.Adadelta(lr=self.initial_lrate),
                        metrics=['accuracy'])

        elif option == "Adam":
            self.model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                        optimizer=keras.optimizers.Adam(lr=self.initial_lrate),
                        metrics=['accuracy'])

        elif option == "SGD":
            self.model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                        optimizer=keras.optimizers.SGD(lr=self.initial_lrate),
                        metrics=['accuracy'])

        elif option == "Nadam":
            self.model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                        optimizer=keras.optimizers.Nadam(lr=self.initial_lrate),
                        metrics=['accuracy'])

        elif option == "RMSprop":
            self.model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                        optimizer=keras.optimizers.Nadam(lr=self.initial_lrate),
                        metrics=['accuracy'])


        return self.model

    def load_model_net(self, path):
        self.model = load_model(path)
        return self.model

    def last_load_model_net(self, experimental_path):
        weights = os.listdir(experimental_path)
        weights = sorted(weights, reverse=True)
        print("[+] WEIGHTS LIST: ", weights)
        print("[+] WEIGHTS RESTORE: ", weights[0])        
        self.model = load_model(os.path.join(experimental_path, weights[0]))
        return self.model
    
    
    def classify_image(self, model, img):
        return model.predict([img])

    def classify_image_from_path(self, model, path, dim):
        img = cv2.imread(path)
        img = cv2.resize(img, (dim, dim))
        img = np.array(img)
        img = img.astype('float32')
        img = img / 255

        img = img.reshape(1, dim, dim, 3)

        print("shape: ", img.shape)
        return model.predict([np.array(img)])

    def classify_bath_image_from_path(self, model, path, clazz, dim):
        cont = [0, 0]
        # print("255")
        for file in os.listdir(path):
            img = cv2.imread(os.path.join(path, file))
            img = cv2.resize(img, (dim, dim))
            cv2.imshow("img", img)
            cv2.waitKey(200)
            img = np.array(img)
            img = img.astype('float32')
            img = img / 255

            img = img.reshape(1, dim, dim, 3)
            # print([np.array(img)])
            # print("shape: ", img.shape)
            vet = model.predict([np.array(img)])
            # print("vet: ", vet)
            if np.argmax(vet[0]) == clazz:
                cont[0] += 1
            else:
                cont[1] += 1
                print(file)
        print("Classe: ", clazz, " Conts: ", cont)


Using TensorFlow backend.


# **`Neural Networks Architectures Keras`**


In [ ]:
import keras.applications as classifiers


class InceptionResNetV2:
    def build_network(self, input_shape, num_classes):
        model = classifiers.inception_resnet_v2.InceptionResNetV2(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class NASNetLarge:
    def build_network(self, input_shape, num_classes):
        model = classifiers.nasnet.NASNetLarge(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class Xception:
    def build_network(self, input_shape, num_classes):
        model = classifiers.xception.Xception(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class InceptionV3:
    def build_network(self, input_shape, num_classes):
        model = classifiers.inception_v3.InceptionV3(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class MobileNet:
    def build_network(self, input_shape, num_classes):
        model = classifiers.mobilenet.MobileNet(input_shape=input_shape,
                                                alpha=1.0,
                                                depth_multiplier=1,
                                                dropout=1e-3,
                                                include_top=True,
                                                weights=None,
                                                input_tensor=None,
                                                pooling='max',
                                                classes=num_classes)
        return model


In [2]:
import keras.applications as classifiers


class MobileNetV2:
    def build_network(self, input_shape, num_classes):
        model = classifiers.mobilenetv2.MobileNetV2(input_shape=input_shape,
                                                alpha=1.0,
                                                include_top=False,
                                                weights=None,
                                                input_tensor=None,
                                                pooling='max',
                                                classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class ResNet50:
    def build_network(self, input_shape, num_classes):
        model = classifiers.resnet50.ResNet50(include_top=True,
                                        input_shape=input_shape,
                                        weights=None,
                                        input_tensor=None,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class VGG16:
    def build_network(self, input_shape, num_classes):
        model = classifiers.vgg16.VGG16(include_top=True,
                                        input_tensor=None,
                                        weights=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [ ]:
import keras.applications as classifiers


class VGG19:
    def build_network(self, input_shape, num_classes):
        model = classifiers.vgg16.VGG16(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


# **`Utils Class`**

In [3]:
import os
import time
import datetime

class Utils:
    def make_dirs(self, path):
        if os.path.exists(path) == False:
            os.makedirs(path)

    def get_date_string(self):
        ts = time.time()
        st = datetime.datetime.fromtimestamp(ts).strftime('%H_%M_%S+%Y-%m-%d')
        return st


# **`Pre Processing Class`**

In [4]:
import cv2
import numpy as np
# from utils.constants import getDataPath
# from utils.constants import getDataAndLabelsPath
# from utils.constants import getLabelsPath
import os

class PreProcessing:

    def load_base_input_experiment(self, path):
        pwd = os.path.join(path, "training")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-data-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-data-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-data.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_data.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-data.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-data.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-data.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-data.npy'))

        x_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)
#         x_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "training")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-label-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-label-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-label.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_labels.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-label.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-label.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-label.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-label.npy'))

        y_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)
#         y_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "testing")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-data-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-data-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-data.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_data.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-data.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-data.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-data.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-data.npy'))

        x_test = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "testing")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-label-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-label-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-label.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_labels.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-label.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-label.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-label.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-label.npy'))

        y_test = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        return x_train, y_train, x_test, y_test


    def load_base(self, path):
        pwd = os.path.join(path, "images")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-data-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-data-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-data.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_data.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-data.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-data.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-data.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-data.npy'))

        images = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "labels")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-label-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-label-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-label.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_labels.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-label.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-label.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-label.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-label.npy'))

        labels = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        return images, labels


    def generate_images_with_size(self, size, save=True, paths=None, path_output=None, faceDetector=None):
        # CIFE_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA = getDataPath()
        if paths == None:
            paths_images = getDataPath()
            path_labels = getLabelsPath()

        total_images = 0

        if len(paths_images) != len(path_labels):
            print("[+] ERROR PATH LABEL AND IMAGE SIZE DIFFERENCE")
            return False

        for index in range(len(paths_images)):
            print("path: ", paths_images[index])
            total_images += self.resize_all_images_label(size, paths_images[index], path_output, path_labels[index], faceDetector, save)

        print("[+] Total Images: ", total_images)
        return True

    # def generate_labels(self, save=False, paths=None, path_output=None):
    #     if paths == None:
    #         paths = getLabelsPath()
    #
    #     labels = []
    #
    #     # for path in enumerate(paths):
    #     for path in paths:
    #         label = np.load(path).tolist()
    #         labels += label
    #
    #     print("[+] Labels len: ", len(labels))
    #
    #     if save == True:
    #         self.make_dirs(os.path.join(path_output, "labels"))
    #         file_name = os.path.join(path_output, "labels",  "labels.npy")
    #         print("[+] File Saving: ", file_name)
    #         np.save(file_name, labels)
    #
    #
    #     return labels

    def load_from_file(self, path):
        return np.load(path)

    def resize_all_images_label(self, size, path_load, path_output, path_label, faceDetector=None, save=True):
        #TODO: Verify others possible resize processing, cubic, linear, etc, upsample
        new_images = []
        new_labels = []

        images = np.load(path_load)
        labels = np.load(path_label)

        print("[+] SIZE: ", size)

        if len(images) != len(labels):
            print("[+] ERROR LABEL AND IMAGE SIZE DIFFERENCE")
            return False

        for index in range(len(images)):
            coordinates = faceDetector.detectCNN(images[index])

            if coordinates != []:
                bbox = coordinates[0]
                # cv2.imshow("img1", images[index])
                # cv2.waitKey(25)
                print(index)
                img = images[index]
                img = img[bbox[0]:bbox[2], bbox[3]:bbox[1]]

                img = cv2.resize(img, (size, size))

                cv2.imshow("img-cropped-RSZ", img)
                cv2.waitKey(10)

                new_labels.append(labels[index])
                new_images.append(img)

        print("resized ", path_load, " :", len(new_images))
        print("labels ", path_label, " :", len(new_labels))

        if save == True:
            base_label = os.path.basename(os.path.normpath(path_load))
            # base_label = base_label.replace(".npy", "")

            self.make_dirs(os.path.join(path_output, "images"))
            file_image_name = os.path.join(path_output, "images", base_label)
            print("[+] File Saving: ", file_image_name)
            np.save(file_image_name, new_images)

            base_label = os.path.basename(os.path.normpath(path_label))
            # base_label = base_label.replace(".npy", "")

            self.make_dirs(os.path.join(path_output, "labels"))
            file_label_name = os.path.join(path_output, "labels",  base_label)
            print("[+] File Saving: ", file_label_name)
            np.save(file_label_name, new_labels)


        return len(new_images)

    def save_images_labels_size(self, size, faceDetector, path_output):
        # images_paths, labels_paths = getDataAndLabelsPath()
        # images = []
        # labels = []
        #
        # # for path in enumerate(paths):
        # for path in labels_paths:
        #     label = np.load(path).tolist()
        #     labels += label
        #     print("len_label: ", len(label))
        # print("len: ", len(labels))

        # self.generate_labels(save=True, path_output=path_output)
        self.generate_images_with_size(size, save=True, faceDetector=faceDetector, path_output=path_output)

    def make_dirs(self, path):
        if os.path.exists(path) == False:
            os.makedirs(path)


# **`Parameters Fine-Tuning`**


In [5]:
COMMENTARY = "teste mobilenetv2 with all databases"

IMG_SIZE = 185

BATCH_SIZE = 35

OPTIMIZERS = ["Adadelta", "SGD", "Nadam", "RMSprop", "Adam"]
OPTIMIZER = OPTIMIZERS[0]

ARCHITECTURES = ["VGG16", "VGG19", "MobileNet", "MobileNetV2", \
                "ResNet50", "InceptionV3", "InceptionResNetV2", "NASNetLarge", \
                "Xception"]
ARCHITECTURE = ARCHITECTURES[4-1]

NUM_CLASSES = 7
EPOCHS = 40

#how many epochs waiting a increasing in validation loss
PATIENCE = 6

#parameters learning rate schedule
INITIAL_LRATE=0.05 
#LIMIT_INF = 0.005
LIMIT_INF = 0.005
DROP=0.75 
EPOCHS_DROP=1.0



# Loading Database

In [6]:
import numpy as np
import os

preProcessing = PreProcessing()


ROOT_INPUT_DATA = "input_data_emotions"
BASE_DIR = "output_"+str(IMG_SIZE)


input_path = os.path.join(ROOT_INPUT_DATA, BASE_DIR)
print("[+] LOADING BASE INPUT")
x_train, y_train, x_test, y_test = preProcessing.load_base_input_experiment(input_path)
print("[+] PERFECT, IT'S LOADED THE INPUT BASE")


[+] LOADING BASE INPUT
[+] PERFECT, IT'S LOADED THE INPUT BASE


# **`Train Script`**


In [ ]:
import tensorflow as tf

utils = Utils()


ROOT_OUTPUT = "mestrado_finish"
EXPERIMENT_ID = "FINAL-" + ARCHITECTURE + "+" + str(IMG_SIZE) + "+" + utils.get_date_string()
output_path = os.path.join(ROOT_OUTPUT, EXPERIMENT_ID)
utils.make_dirs(output_path)
# output_path = drive.create_dir(top=ROOT_OUTPUT, name=EXPERIMENT_ID)


common = Common(INITIAL_LRATE, DROP, EPOCHS_DROP, LIMIT_INF)

common.write_file_info(output_path, ["architecture: " + ARCHITECTURE, "img_size: " + str(IMG_SIZE)])

input_shape = (IMG_SIZE, IMG_SIZE, 3)

#NORMALIZE IMAGES DATASET
# x_train = x_train.astype('float32')
# x_train /= 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

print("[+] TYPE NDARRAY: ", x_train.dtype)

with tf.device('/gpu:0'):
    print("[+] " + ARCHITECTURE)
    if ARCHITECTURE == "VGG16":
        net = VGG16()
    elif ARCHITECTURE == "VGG19":
        net = VGG19()
    elif ARCHITECTURE == "MobileNet":
        net = MobileNet()
    elif ARCHITECTURE == "MobileNetV2":
        net = MobileNetV2()
    elif ARCHITECTURE == "ResNet50":
        net = ResNet50()
    elif ARCHITECTURE == "InceptionV3":
        net = InceptionV3()
    elif ARCHITECTURE == "InceptionResNetV2":
        net = InceptionResNetV2()
    elif ARCHITECTURE == "NASNetLarge":
        net = NASNetLarge()
    elif ARCHITECTURE == "Xception":
        net = Xception()


        
    print("[+] BUILD NET")
    model = net.build_network(input_shape, NUM_CLASSES)

    print("[+] COMPILE")
    model = common.compile_net(model, output_path, OPTIMIZER)

    print("[+] ADD COMMENTARY")
    common.addCommentary(output_path, COMMENTARY)
    
    print("[+] TRAIN")
    model, history = common.train(model, x_train, y_train, x_test, y_test, BATCH_SIZE, EPOCHS, output_path, PATIENCE)

    print("[+] SAVE ON HISTORY")
    common.save_plot_losses(history, output_path)
    common.save_plot_acc(history, output_path)

x_train shape: (38868, 185, 185, 3)
38868 train samples
[+] TYPE NDARRAY:  uint8
[+] MobileNetV2
[+] BUILD NET
[+] COMPILE
[+] OPTIMIZER:  Adadelta
[+] ADD COMMENTARY
[+] TRAIN
[+] BATCH_SIZE:  35
Train on 38868 samples, validate on 12961 samples
Epoch 1/40
[+] initial:  0.05
[+] ANTIGO K:  0.05
[+] NOVO:  0.05
38868/38868 [==============================] - 379s 10ms/step - loss: 4.4573 - acc: 0.1480 - val_loss: 3.4524 - val_acc: 0.2610

Epoch 00001: val_loss improved from inf to 3.45237, saving model to mestrado_finish/FINAL-MobileNetV2+185+22_14_42+2019-04-22/weights.01-3.45.hdf5
Epoch 2/40
[+] initial:  0.05
[+] ANTIGO K:  0.05
[+] NOVO:  0.028125
38868/38868 [==============================] - 362s 9ms/step - loss: 2.0208 - acc: 0.2902 - val_loss: 2.0428 - val_acc: 0.2986

Epoch 00002: val_loss improved from 3.45237 to 2.04278, saving model to mestrado_finish/FINAL-MobileNetV2+185+22_14_42+2019-04-22/weights.02-2.04.hdf5
Epoch 3/40
[+] initial:  0.05
[+] ANTIGO K:  0.028125
[+] NOVO

# Restore model and Train

In [ ]:
import tensorflow as tf

common = Common()

RECOVERY_PATH = "InceptionResNetV2+185+19_57_04+2018-12-04"


ROOT_OUTPUT = "/content/drive/My Drive/mestrado_finish/experimentos"

output_path = os.path.join(ROOT_OUTPUT, RECOVERY_PATH)

#NORMALIZE IMAGES DATASET
# x_train = x_train.astype('float32')
# x_train /= 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

print("[+] TYPE NDARRAY: ", x_train.dtype)

with tf.device('/gpu:0'):
#     print("[+] " + ARCHITECTURE)
#     if ARCHITECTURE == "VGG16":
#         net = VGG16()
#     elif ARCHITECTURE == "VGG19":
#         net = VGG19()
#     elif ARCHITECTURE == "MobileNet":
#         net = MobileNet()
#     elif ARCHITECTURE == "MobileNetV2":
#         net = MobileNetV2()
#     elif ARCHITECTURE == "ResNet50":
#         net = ResNet50()
#     elif ARCHITECTURE == "InceptionV3":
#         net = InceptionV3()
#     elif ARCHITECTURE == "InceptionResNetV2":
#         net = InceptionResNetV2()


    print("[+] RESTORE LAST MODEL")
    model = common.last_load_model_net(output_path)

    print("[+] COMPILE")
    model = common.compile_net(model, output_path, OPTIMIZER)

    print("[+] TRAIN")
    model, history = common.train(model, x_train, y_train, x_test, y_test, BATCH_SIZE, EPOCHS, output_path, PATIENCE)

    print("[+] SAVE ON HISTORY")
    common.save_plot_losses(history, output_path)
    common.save_plot_acc(history, output_path)